In [5]:
from pathlib import Path
root_dir = Path('.').resolve().parent
import sys
sys.path.append(root_dir)

%load_ext autoreload
%autoreload 2

In [11]:
import torch
from scipy.io import wavfile
from scipy import signal
import numpy as np

In [14]:
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))# nperseg = 320
    noverlap = int(round(step_size * sample_rate / 1e3))# noverlap = 160
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)
def convert(filename: str)->torch.FloatTensor:
    """
    Convert wav file to feature
    """
    sample_rate, samples = wavfile.read(filename)
    print(samples.shape)
    freqs, times, spectrogram = log_specgram(samples, sample_rate)
    print(spectrogram.shape)
    return torch.tensor(spectrogram)
convert(root_dir/ 'datasets'/ 'speech_commands_v1'/ 'bed'/ '1aed7c6d_nohash_0.wav')

(16000,)
(99, 161)


tensor([[ -3.7267,  -3.4231,  -0.3736,  ..., -11.8554, -10.8581, -11.5257],
        [ -5.6856,  -4.1784,  -2.0797,  ..., -11.4320, -11.4188, -12.1216],
        [ -6.1031,  -5.6279,   0.2300,  ..., -11.5894, -11.5558, -13.0355],
        ...,
        [ -9.1891,  -5.4730,  -2.6923,  ..., -11.7991, -13.1026, -12.1093],
        [ -9.6067,  -4.5401,  -2.6894,  ..., -12.9995, -11.7861, -13.6048],
        [ -9.0516,  -4.6354,  -1.5045,  ..., -10.9258, -17.5748, -16.7621]])